In [ ]:
#r "B:\BoSSS-smuda\public\src\L4-application\BoSSSpad\bin\Release\net5.0\BoSSSpad.dll"
//#r "BoSSSpad.dll"
using System;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Solution;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.BoSSSpad;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Control;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.NSECommon;
using BoSSS.Application.XNSE_Solver.LoadBalancing;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.XdgTimestepping;

using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [ ]:
/*
// Is used at Jenkins to generate individual names (for output .json)
string dbname = System.Environment.GetEnvironmentVariable("DATABASE_NAME");
string buildname = System.Environment.GetEnvironmentVariable("JOB_NAME");
//defaults
buildname = String.IsNullOrEmpty(buildname)? "Benchmark" : buildname;
//string thedate = $"{System.DateTime.Today.Day}-{System.DateTime.Today.Month}-{System.DateTime.Today.Year}";
dbname = String.IsNullOrEmpty(dbname)? "dbname" : dbname;
string table_name = String.Concat(buildname, "_", dbname);
*/

In [ ]:
string table_name = "Stokes_Channel_pTG_studyHPC";

In [ ]:
ExecutionQueues

In [ ]:
var myBatch = (MsHPC2012Client)ExecutionQueues[2];
/*
var AddSbatchCmds = new List<string>();
AddSbatchCmds.AddRange(new string[]{"#SBATCH -C avx512", "#SBATCH --mem-per-cpu="+2000});
myBatch.AdditionalBatchCommands = AddSbatchCmds.ToArray();
myBatch.AdditionalBatchCommands
*/

In [ ]:
string WFlowName = table_name;
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.Init(WFlowName);
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
var myDB = BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.AllDatabases.Pick(2); myDB

In [ ]:
static class Utils {
    // DOF per cell in 3D
    public static int Np(int p) {
        return (p*p*p + 6*p*p + 11*p + 6)/6;
    }    
    
    //Non-equidistant nodes
    public static double[] SinLinSpacing(double l, double r, double a, int n) {
        double[] linnodes = GenericBlas.Linspace(-Math.PI * 0.5, Math.PI * 0.5, n);
        double[] linnodes2 = GenericBlas.Linspace(-1, 1, n);
        double[] nodes = new double[n];

        for (int i = 0; i < n; i++)
            //nodes[i] = linnodes2[i] * (1 - a) + (1.0 - Math.Sin(linnodes[i])) * a;
            nodes[i] = linnodes2[i] * (1 - a) + Math.Sin(linnodes[i])*a;

        for (int i = 0; i < n; i++)
            nodes[i] = nodes[i] * (r - l)*0.5 + l;
        return nodes;
    }
}

In [ ]:

int[] Resolutions_3D = new int[] { 4 };
//int[] Resolutions_3D = new int[] { 2 };
IGridInfo[] grids = new IGridInfo[Resolutions_3D.Length];
for(int cnt = 0; cnt < Resolutions_3D.Length; cnt++) {
    int Res = Resolutions_3D[cnt];    
    
    double[] xNodes = GenericBlas.Linspace(0, 10, Res + 1);
    double[] yNodes = GenericBlas.Linspace(-1, +1, Res + 1);
    double[] zNodes = GenericBlas.Linspace(-1, +1, Res + 1);
    int J = (xNodes.Length - 1)*(yNodes.Length - 1)*(zNodes.Length - 1);
    string prjname = BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.CurrentProject;
    string GridName = string.Format(prjname + "_J" + J);
    
    try{
        grids[cnt]=null;
        foreach(IGridInfo grd in myDB.Grids){
            bool check = grd.Name.Contains(string.Format("_J"+ J));
            if(check){
            grids[cnt]=grd;
            }
        }
    } catch (Exception ex){
        //Console.WriteLine(ex.Message);
    }
    
    
    //grids[cnt] = tempDB.Grids.SingleOrDefault(grd => grd.Name == GridName);
    
    if(grids[cnt] == null){
        Console.WriteLine("Creating grid with " + J + " cells.");
        
        GridCommons g;
        g      = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);
        g.Name = GridName;
        
        g.EdgeTagNames.Add(1, "Velocity_inlet");
        g.EdgeTagNames.Add(2, "Wall");
        g.EdgeTagNames.Add(3, "Pressure_Outlet");

        g.DefineEdgeTags(delegate (double[] _X) {
            var X = _X;
            double x, y, z;
            x = X[0];
            y = X[1];
            z = X[2];
            if (Math.Abs(x - 0) < 1E-8)
                return 1;
            if (Math.Abs(x - 10) < 1E-8)
                return 3;
            return 2;
        });
      
        myDB.SaveGrid(ref g);  
        grids[cnt] = g;
    } else {
        Console.WriteLine("Found Grid: " + grids[cnt]);
        if(grids[cnt].NumberOfCells != J)
            throw new Exception("J mismatch");
        
        if(grids[cnt].SpatialDimension != 3)
            throw new Exception("D mismatch");
    }
}

In [ ]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.Timestepping;
using BoSSS.Solution.XdgTimestepping ;

In [ ]:
Func<LinearSolverCode, int,int, IGridInfo, int ,XNSE_Control> GenerateChannel = delegate (LinearSolverCode solver, int p, int pLow, IGridInfo grid, int DOF){
    
    // ProjectName
    // ============
    int J         = grid.NumberOfCells;
    
    string caseName = string.Format($"J{J}_p{p}_{solver}_{pLow}");
    Console.WriteLine("setting up: " + caseName);
    
    XNSE_Control C       = new XNSE_Control();
    C.SetDatabase(myDB);
    C.savetodb           = true;
    C.ProjectName        = WFlowName;
    C.ProjectDescription = "static droplet";
    C.SessionName        = caseName;
    C.GridGuid           = grid.ID;
    
    // DG degrees
    // ============
    
    C.FieldOptions.Add("Velocity*", new FieldOpts() {
        Degree = p,
        SaveToDB = FieldOpts.SaveToDBOpt.TRUE
    });
    C.FieldOptions.Add("Pressure", new FieldOpts() {
        Degree = p - 1,
        SaveToDB = FieldOpts.SaveToDBOpt.TRUE
    });
    C.FieldOptions.Add("PhiDG", new FieldOpts() {
        SaveToDB = FieldOpts.SaveToDBOpt.TRUE
    });
    C.FieldOptions.Add("Phi", new FieldOpts() {
        Degree = 2,
        SaveToDB = FieldOpts.SaveToDBOpt.TRUE
    });
    C.FieldOptions.Add("Curvature", new FieldOpts() {
        Degree = Math.Max(4, 2*p + 2),
        SaveToDB = FieldOpts.SaveToDBOpt.TRUE
    });
    
    // Phys. Parameters
    // ============
    
    C.PhysicalParameters.rho_A             = 1e-3; //     kg / cm³
    C.PhysicalParameters.rho_B             = 1.2e-6; //   kg / cm³
    C.PhysicalParameters.mu_A              = 1e-5; //      kg / cm / sec
    C.PhysicalParameters.mu_B              = 17.1e-8; //   kg / cm / sec
    C.PhysicalParameters.IncludeConvection = false;
    C.PhysicalParameters.Material          = true;
    
    C.LSContiProjectionMethod = ContinuityProjectionOption.None;
    C.CutCellQuadratureType   = BoSSS.Foundation.XDG.XQuadFactoryHelper.MomentFittingVariants.Saye;
    
    // Solver Stuff
    // ============
    
    C.UseSchurBlockPrec                = true;
    C.LinearSolver.NoOfMultigridLevels = 5;
    C.LinearSolver.MaxSolverIterations = 10;
    C.LinearSolver.pMaxOfCoarseSolver  = pLow;
    
    C.LinearSolver.TargetBlockSize = Math.Min(J*DOF/2,10000); //Make at least two blocks with Schwarz plz...
    //C.LinearSolver.TargetBlockSize = (int)(J*DOF/(NoSblocks-1));

    C.LinearSolver.ConvergenceCriterion = 1e-12;
    C.LinearSolver.SolverCode           = solver;
    C.LinearSolver.verbose              = true;
    C.NonLinearSolver.verbose           = true;

    C.Option_LevelSetEvolution                          = LevelSetEvolution.None;
    C.AdvancedDiscretizationOptions.ViscosityMode       = ViscosityMode.FullySymmetric;

    double VelocityIn = 1.0;
    C.AddInitialValue(VariableNames.LevelSetCGidx(0), new Formula("X => -1"));
    C.AddBoundaryValue("Velocity_inlet", "VelocityX", new Formula($"(X) => {VelocityIn}"));
    C.AddInitialValue("VelocityX", new Formula($"(X) => {VelocityIn}"));
    C.AddInitialValue("Pressure", new Formula(@"X => 0"));
    C.AddBoundaryValue("Pressure_Outlet");

    // Timestepping
    // ============

    C.TimeSteppingScheme           = TimeSteppingScheme.ImplicitEuler;
    C.Timestepper_LevelSetHandling = LevelSetHandling.None;
    C.TimesteppingMode             = AppControl._TimesteppingMode.Steady;
    C.dtFixed                      = double.MaxValue;
            
    return C;
    };

In [ ]:
int[] PolyDegS = new int[] {4};
//int[] PolyDegS = new int[] {2};

In [ ]:
// Solvers which we want to instrument:
LinearSolverCode[] solver_nameS = new LinearSolverCode[] {
    LinearSolverCode.exp_pTG
}; 

In [ ]:
//List<IGridInfo> t_grids=new List<IGridInfo>();
//grids.Pick(1,2,3,4,5,6,7).ForEach(g => t_grids.Add(g));
//t_grids.ToArray();
IGridInfo[] t_grids = grids;
t_grids

In [ ]:
// MaxDof for one claculation (we skip fine grids for higher polynomial orders)
int MaxDof = 5000000;

In [ ]:
using BoSSS.Solution.XNSECommon;
using BoSSS.Foundation.XDG;

In [ ]:
List<XNSE_Control> controls = new List<XNSE_Control>();
controls.Clear();
foreach(LinearSolverCode solver_name in solver_nameS) {
foreach(int k in PolyDegS) {
for(int pLow=k;pLow>=0;pLow--)
foreach(IGridInfo grd in t_grids) {

    int Np_V = Utils.Np(k);
    int Np_p = Utils.Np(k-1);
    int J    = grd.NumberOfCells;
    int DOF  = Np_V*3+Np_p;
    if(J*DOF > MaxDof)
        continue;
    
    var Ctrl = GenerateChannel(solver_name,k,pLow,grd,DOF);
    
    controls.Add(Ctrl);
}
}
}

In [ ]:
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.ResetSessionsCache();

In [ ]:
var nodes = new List<string>();
nodes.Add("HPCCLUSTER");
nodes.Add("HPCLUSTER3");
myBatch.ComputeNodes = nodes.ToArray();

In [ ]:
foreach(var ctrl in controls) {
    Console.WriteLine(" Submitting: " + ctrl.SessionName); 
    var aJob = new Job(ctrl.SessionName, typeof(XNSE));
    aJob.SetControlObject(ctrl);
    aJob.NumberOfMPIProcs         = 1;
    aJob.ExecutionTime            = "1:00:00";
    aJob.UseComputeNodesExclusive = true;
    aJob.Activate(myBatch);
}